# eICU database Analysis:

Documentation here: https://eicu-crd.mit.edu/about/eicu/

comrpbar sensitive el primero y seguir comprobando si sensitive + non-sensitive coge el sensitive

# Environment

In [ ]:
from google.colab import drive
from google.colab import files
import gzip
import pandas as pd
import json
import sys
import numpy as np
import plotly.express as px
from scipy import stats
from sklearn.impute import SimpleImputer

In [ ]:
np.set_printoptions(threshold=sys.maxsize)

# Loading extracting the data


### Patients data

### Microbiology tests

### Merge raw patient microlab+organism




In [ ]:
patient_microbiology_organism_df = patient_df.merge(microbiology_df, how='inner', on='patientunitstayid')

# Exclusion criteria

## Deletion of rows with NaNs in organism, antibiotic and sensitivitylevel

In [ ]:
patient_microbiology_organism_df = patient_microbiology_organism_df.dropna(subset=['organism','antibiotic','sensitivitylevel'])


# New variables creation

## Sensitivity Level

In [ ]:
patient_microbiology_organism_df.loc[patient_microbiology_organism_df['sensitivitylevel'] == 'Sensitive', 'sensitivitylevel'] = 'No resistant'

In [ ]:
patient_microbiology_organism_df.loc[patient_microbiology_organism_df['sensitivitylevel'] == 'Intermediate', 'sensitivitylevel'] = 'No resistant'

In [ ]:
patient_microbiology_organism_df['sensitivitylevel'].unique()

array(['No resistant', 'Resistant'], dtype=object)

# Inclusion criteria

### At least 16 years old


In [ ]:
patient_microbiology_organism_df.loc[patient_microbiology_organism_df['age'] == '> 89', 'age'] = 90

In [ ]:
patient_microbiology_organism_df['age'] = patient_microbiology_organism_df['age'].astype(int)

<ipython-input-175-67f93aa838d1>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
patient_microbiology_organism_df = patient_microbiology_organism_df.loc[patient_microbiology_organism_df['age'] >= 16]

## Filter patients by Blocks



### Block 1: BLEE-producing Enterobacteriaceae. This cannot be known in many cases and we assume those that are R to cefotaxime and Cetriaxone and ceftazidime and Cepefime.

E coli
Klebsiella pneumoniae
Enterobacter cloacae
Enterobacter sp
Klebsiella oxytoca
Proteus mirabilis

In [ ]:
antibiotics_block1_list=['cefepime','ceftriaxone','cefotaxime','ceftazidime']
organism_block1_list = ['Escherichia coli','Klebsiella pneumoniae','Enterobacter sp.','Enterobacter cloacae','Klebsiella oxytoca','Proteus mirabilis (indole +)']

In [ ]:
block1 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block1_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block1_list))]

In [ ]:
block1['Bloque'] = 'Bloque 1: BLEE'

<ipython-input-183-0ae769a3cc6f>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Block 2: Carbapenemase-producing Enterobacteriaceae.This cannot be known in many cases and we assume those that are R to meropenem 3 imipenem

E coli
Klebsiella pneumoniae
Enterobacter cloacae
Enterobacter sp
Klebsiella oxytoca
Proteus mirabilis

In [ ]:
antibiotics_block2_list=['meropenam','imipenem/cilastatin']
organism_block2_list = ['Escherichia coli','Klebsiella pneumoniae','Enterobacter sp.','Enterobacter cloacae','Klebsiella oxytoca','Proteus mirabilis (indole +)']

In [ ]:
block2 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block2_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block2_list))]

In [ ]:
block2['Bloque'] = 'Bloque 2: Carbapenemasas'

<ipython-input-186-61713886cc95>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Block 3: Carbapenem-resistant (meropenem and imipenem)

Pseudomonas aeruginosa
Acinetobacter baumannii


In [ ]:
antibiotics_block3_list=['meropenam','imipenem/cilastatin']
organism_block3_list = ['Pseudomonas aeruginosa','Acinetobacter baumanii']

In [ ]:
block3 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block3_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block3_list))]

In [ ]:
block3['Bloque'] = 'Bloque 3: Carbapenémicos'

<ipython-input-189-af20fdf4c848>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Block 4: Trimethropin/sulfacetozaxol resistant

Stenotrophomonas maltophila


In [ ]:
antibiotics_block4_list=['trimethoprim/sulfamethoxazole']
organism_block4_list = ['Stenotrophomonas maltophila']

In [ ]:
block4 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block4_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block4_list))]

In [ ]:
block4['Bloque'] = 'Bloque 4: trimetropin/sulfacetozaxol'

<ipython-input-192-cf47dd10f4fc>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Bloque 5: Vancomycin-resistant

Enterococcus faecium

In [ ]:
antibiotics_block5_list=['vancomycin']
organism_block5_list = ['Enterococcus fecium']

In [ ]:
block5 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block5_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block5_list))]

In [ ]:
block5['Bloque'] = 'Bloque 5: vancomicina'

<ipython-input-195-e79c79a71036>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Bloque 6: Oxacillin-resistant
Staphylococcus aureus

In [ ]:
antibiotics_block6_list=['oxacillin']
organism_block6_list = ['Staphylococcus aureus']

In [ ]:
block6 = patient_microbiology_organism_df.loc[(patient_microbiology_organism_df['antibiotic'].isin(antibiotics_block6_list)) &  (patient_microbiology_organism_df['organism'].isin(organism_block6_list))]

In [ ]:
block6['Bloque'] = 'Bloque 6: oxacilina'

<ipython-input-198-cb2588ca69cf>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



### Union of all blocks in a single DF

In [ ]:
patient_microbiology_organism_df_filtered = pd.concat([block1,block2, block3, block4, block5, block6])


#### Plot results

In [ ]:
patient_microbiology_organism_df_filtered_count_by_block = patient_microbiology_organism_df_filtered['Bloque'].value_counts(dropna=False).to_frame().reset_index()

In [ ]:
fig = px.bar(patient_microbiology_organism_df_filtered_count_by_block, x='Bloque', y='index', text_auto=True)
fig.show()

### Organism-antibiotic count

In [ ]:
patient_microbiology_organism_df_filtered['organism-antibiotic'] = patient_microbiology_organism_df_filtered['organism']+ '-' + patient_microbiology_organism_df_filtered['antibiotic']

In [ ]:
patient_microbiology_organism_df_filtered_count_by_org_antib = patient_microbiology_organism_df_filtered['organism-antibiotic'].value_counts(dropna=False).to_frame().reset_index()

#### Plot results

In [ ]:
fig = px.bar(patient_microbiology_organism_df_filtered_count_by_org_antib, x='organism-antibiotic', y='index', text_auto=True)
fig.show()

### Eliminating repeated rows by ICU stay.

In [ ]:
df_resultado_count_by_block = df_resultado[['Bloque','sensitivitylevel']].value_counts(dropna=False).to_frame().reset_index()

In [ ]:
df_resultado_count_by_block_o = df_resultado[['Bloque']].value_counts(dropna=False).to_frame().reset_index()

In [ ]:
len(list(df_resultado['patientunitstayid'].unique()))

312

# Final Histogram and number of entries by block

In [ ]:
df_resultado_count_by_block_o

,Bloque,0
0,Bloque 1: BLEE,367
1,Bloque 6: oxacilina,114
2,Bloque 2: Carbapenemasas,91
3,Bloque 3: Carbapenémicos,39
4,Bloque 5: vancomicina,8
5,Bloque 4: trimetropin/sulfacetozaxol,4


In [ ]:
df_resultado_count_by_block

,Bloque,sensitivitylevel,0
0,Bloque 1: BLEE,No resistant,315
1,Bloque 2: Carbapenemasas,No resistant,89
2,Bloque 6: oxacilina,Resistant,58
3,Bloque 6: oxacilina,No resistant,56
4,Bloque 1: BLEE,Resistant,52
5,Bloque 3: Carbapenémicos,No resistant,32
6,Bloque 3: Carbapenémicos,Resistant,7
7,Bloque 4: trimetropin/sulfacetozaxol,No resistant,4
8,Bloque 5: vancomicina,No resistant,4
9,Bloque 5: vancomicina,Resistant,4


In [ ]:
fig = px.bar(df_resultado_count_by_block, x='Bloque', y=0, color="sensitivitylevel", text_auto=True)
fig.show()

In [ ]:
patient_microbiology_organism_df_filtered_count_by_org_antib = df_resultado[['organism-antibiotic','sensitivitylevel']].value_counts(dropna=False).to_frame().reset_index()

In [ ]:
patient_microbiology_organism_df_filtered_count_by_org_antib

,organism-antibiotic,sensitivitylevel,0
0,Escherichia coli-ceftriaxone,No resistant,69
1,Staphylococcus aureus-oxacillin,Resistant,58
2,Staphylococcus aureus-oxacillin,No resistant,56
3,Escherichia coli-cefepime,No resistant,48
4,Klebsiella pneumoniae-ceftriaxone,No resistant,45
5,Escherichia coli-ceftazidime,No resistant,37
6,Klebsiella pneumoniae-cefepime,No resistant,32
7,Escherichia coli-imipenem/cilastatin,No resistant,26
8,Klebsiella pneumoniae-ceftazidime,No resistant,22
9,Escherichia coli-meropenam,No resistant,21


In [ ]:
import plotly.io as pio


In [ ]:
fig = px.bar(patient_microbiology_organism_df_filtered_count_by_org_antib, x='organism-antibiotic', y=0, color="sensitivitylevel",text_auto=True)
fig.update_layout(
    autosize=False,
    width=1100,
    height=1100,
)
fig.show()

In [ ]:
pio.show(fig)